In [121]:

from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, KFold, cross_val_predict, \
    cross_validate
import xgboost as xgb


with open('pss.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
password = lines[0]
db = lines[1]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database=db,
    use_pure=True
)

my_cursor = mydb.cursor()
steps_title_list = []
d_steps_title = {}
university_name_list = []
d_uni_name = {}

# def convert_nans(df):
#     # job_applicant_id,gender,age,marriage_status,language,degree,skill,num_prev_company,work_interval,steps_title
#     df['steps_title'] = df['steps_title'].fillna(6)
#     df['age'] = df['age'].fillna(-1)
#     df['skill'] = df['skill'].fillna(-1)
#     df['marriage_status'] = df['marriage_status'].fillna(-1)
#     # df['language'] = df['language'].fillna(-1)
#     # df['job_contract_type'] = df['job_contract_type'].fillna(-1)
#     df['gender'] = df['gender'].fillna(-1)
#     df['degree'] = df['degree'].fillna(-1)
#     df['skill'] = df['skill'].fillna(-1)
#     df['num_prev_company'] = df['num_prev_company'].fillna(-1)
#     df['work_interval'] = df['work_interval'].fillna(-1)





my_cursor.execute('select distinct steps_title from useful_data')
for item in my_cursor:
    steps_title_list.append(item)


for title in steps_title_list:
    if title[0] == 'نیازمند تعیین وضعیت':
        d_steps_title[title[0]] = 0
    elif title[0] == 'تایید برای مصاحبه':
        d_steps_title[title[0]] = 1
    elif title[0] == 'استخدام شده':
        d_steps_title[title[0]] = 2
    elif title[0] == 'رد شده':
        d_steps_title[title[0]] = 3
    elif title[0] == 'انصراف از مصاحبه':
        d_steps_title[title[0]] = 4
    else:
        d_steps_title[title[0]] = 6


df_train = pd.read_csv("train.csv")
df_train.head(10)
df_test = pd.read_csv("test.csv")
df_train['steps_title'] = df_train['steps_title'].map(d_steps_title)
df_test['steps_title'] = df_test['steps_title'].map(d_steps_title)
# convert_nans(df_train)
# convert_nans(df_test)
# [2 , 4 , 5 , 6 , 7 ,10]
x_train =df_train[df_train.columns[1:11]]
x_test = df_test[df_test.columns[1:11]]
y_test = df_test['steps_title']
y_train = df_train['steps_title']
kfold = KFold(n_splits=10)
features_train = df_train.columns[1:11]
# clf = RandomForestClassifier(n_jobs=2 , random_state=0)
clf = xgb.XGBClassifier(seed=42, subsample=0.9)
cv_results = cross_val_score(clf,x_train , y_train,cv=kfold , scoring='accuracy')
output = cross_validate(clf, x_train, y_train, cv=10, scoring = 'accuracy', return_estimator =True)
y_pred = cross_val_predict(clf,x_test,y_test,cv=10)
proba = cross_val_predict(clf,x_test,y_test,cv=10, method='predict_proba')
clf.fit(x_train , y_train , verbose=0, eval_set =[(x_test, y_test)])
# clf.fit(x_train , y_train)
results = (cv_results.mean(),cv_results.std())
print(results)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(pd.crosstab(y_test,y_pred))
df_test["prediction"] = y_pred
# for idx,estimator in enumerate(output['estimator']):
#     print("Features sorted by their score for estimator {}:".format(idx))
#     feature_importances = pd.DataFrame(estimator.feature_importances_,
#                                         index= features_train,
#                                         columns=['importance']).sort_values('importance', ascending=False)
#     print(feature_importances)


proba_0 = []
proba_1 = []
type(proba)
for prob0 , prob1 in proba:
    proba_0.append(float(prob0))
    proba_1.append(float(prob1))
df_test['Prob_0'] = proba_0
df_test['Prob_1'] = proba_1
final_df = df_test.sort_values(by=['Prob_0'], ascending=False)
print(final_df.loc[final_df['steps_title'] == final_df['prediction'] , 'job_applicant_id'].iloc[0:9])
# print(final_df.loc['job_applicant_id'].iloc[0:9])
#
final_df.head(50)

/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  Note
/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  Note
/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label e

(0.9756786051125674, 0.00443660789743655)
accuracy :  0.7712765957446809
col_0         2   3
steps_title        
2            74  20
3            23  71
112    44253
28     47259
11     46145
39     46135
31      3665
145    46207
152     3419
174     8831
53     46335
Name: job_applicant_id, dtype: int64


,job_applicant_id,gender,age,marriage_status,languages,contract_type,degree,average_gpa,skill,num_prev_company,work_interval,steps_title,prediction,Prob_0,Prob_1
112,44253,0.0,41.0,NaN,0,0,NaN,NaN,1,1,0,2,2,0.997357,0.002643
28,47259,0.0,24.0,NaN,0,0,NaN,NaN,1,1,0,2,2,0.994886,0.005114
177,9280,NaN,27.0,NaN,0,0,NaN,NaN,1,2,4,3,2,0.994599,0.005402
11,46145,0.0,30.0,NaN,0,0,NaN,NaN,1,1,0,2,2,0.993616,0.006384
39,46135,0.0,35.0,NaN,0,0,NaN,NaN,1,1,0,2,2,0.992919,0.007081
31,3665,NaN,32.0,NaN,0,0,NaN,NaN,1,1,0,2,2,0.991989,0.008011
33,29650,NaN,36.0,NaN,0,1,NaN,NaN,1,1,0,3,2,0.991203,0.008797
145,46207,0.0,37.0,NaN,0,0,NaN,NaN,1,1,0,2,2,0.990516,0.009484
152,3419,NaN,35.0,NaN,1,0,NaN,NaN,1,1,1,2,2,0.987167,0.012833
174,8831,0.0,NaN,NaN,0,0,4.0,17.700,1,1,0,2,2,0.986791,0.013209
